### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [59]:
district_summary = pd.DataFrame([{    
    "Total Schools" : len(school_data_complete["school_name"].unique()),
    "Total Students" : "{:,.0f}".format(len(school_data_complete["student_name"])),
    "Total Budget" : "${:,.2f}".format(school_data["budget"].sum()),
    "Average Math Score" : "{:.6f}".format(student_data["math_score"].mean()),
    "Average Reading Score" : "{:.5f}".format(student_data["reading_score"].mean()),
    "% Passing Math" : "{:.6f}".format((len(student_data[student_data["math_score"] >= 70]["math_score"])/len(school_data_complete["student_name"]))*100),
    "% Passing Reading" : "{:.6f}".format((len(student_data[student_data["reading_score"] >= 70]["reading_score"])/len(school_data_complete["student_name"]))*100),
    "% Overall Passing" : "{:.6f}".format((len(student_data[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70)]["reading_score"])/len(school_data_complete["student_name"]))*100) 
}])

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

Add "Per Student Budget" column to the dataframe

In [60]:
school_data["Per Student Budget"] = school_data["budget"]/school_data["size"]


Calculate average maths and reading scores for each school

In [61]:
student_data_df = student_data[["school_name","math_score","reading_score"]]
student_data_group_df = student_data_df.groupby(['school_name'])
student_data_mean_df = student_data_group_df.mean()

Calculate "% Passing Reading"

In [62]:
student_data_df1 = student_data_df.groupby(["school_name"]).count()
total_students = student_data_df1["reading_score"]

student_data_df2 = student_data_df.loc[student_data_df.reading_score >= 70].groupby(["school_name"]).count()
reading_pass = student_data_df2["reading_score"]

reading_percentage_df = pd.DataFrame((reading_pass/total_students)*100)
reading_percentage_df = reading_percentage_df.rename(columns = {
    "reading_score":"% Passing Reading"
    })

Calculate "% Passing Math"

In [63]:
total_students = student_data_df1["math_score"]
student_data_df3 = student_data_df.loc[student_data_df.math_score >= 70].groupby(["school_name"]).count()
math_pass = student_data_df3["math_score"]

math_percentage_df = pd.DataFrame((math_pass/total_students)*100)

math_percentage_df = math_percentage_df.rename(columns = {
    "math_score":"% Passing Math"
    })


Calculate "% Overall Passing"

In [89]:
total_students = student_data_df1["math_score"]
student_data_df4 = student_data_df.loc[(student_data_df.math_score >= 70) & (student_data_df.reading_score >= 70)].groupby(["school_name"]).count()
overall_pass = student_data_df4["math_score"]

overall_percentage_df = pd.DataFrame((overall_pass/total_students)*100)
overall_percentage_df = overall_percentage_df.rename(columns = {
    "math_score":"% Overall Passing"
    })

Combine all components by merging each data frame

In [90]:
school_summary_df = pd.merge(school_data,student_data_mean_df,on="school_name")
del school_summary_df["School ID"]
school_summary_df = pd.merge(school_summary_df,math_percentage_df,on="school_name")
school_summary_df = pd.merge(school_summary_df,reading_percentage_df,on="school_name")
school_summary_df = pd.merge(school_summary_df,overall_percentage_df,on="school_name")
school_summary_df = school_summary_df.sort_values(by=['school_name'])
school_summary_df.style.hide_index()
school_summary_df = school_summary_df.rename(columns = {
    "math_score":"Average Math Score",
    "reading_score":"Average Reading Score", 
    "type":"School Type",
    "size" : "Total Students",
    "budget": "Total School Budget",
    "school_name": ""
    })

def bold_cols(s):
    return 'font-weight:bold' 

school_summary_formatted_df = school_summary_df.copy()

#formatting of the report

format_dict = {
    "Total School Budget":"${:,.2f}", 
    "Per Student Budget": "${:,.2f}", 
    "Average Math Score": "{:,.6f}",
    "Average Reading Score": "{:,.6f}",
    "% Passing Math": "{:,.6f}",
    "% Passing Reading": "{:,.6f}",
    "% Overall Passing": "{:,.6f}"
    }
               
school_summary_formatted_df =school_summary_formatted_df.style.format(format_dict).hide_index().applymap(bold_cols, subset=pd.IndexSlice[:, ['']])
school_summary_formatted_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [87]:
top_overall_df = school_summary_df.copy()
top_overall_df = top_overall_df.sort_values("% Overall Passing", ascending=False)
top_overall_df = top_overall_df.head(5).style.format(format_dict).hide_index().applymap(bold_cols, subset=pd.IndexSlice[:, ['']])
top_overall_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [86]:
bottom_overall_df = school_summary_df.copy()
bottom_overall_df = bottom_overall_df.sort_values("% Overall Passing")
bottom_overall_df = bottom_overall_df.head(5).style.format(format_dict).hide_index().applymap(bold_cols, subset=pd.IndexSlice[:, ['']])
bottom_overall_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [91]:
student_data_math_df = student_data[['school_name','grade','math_score']]
student_data_math_grouped_9th_df = student_data_math_df.loc[(student_data_math_df.grade == '9th')].groupby(['school_name'])["math_score"].mean()
student_data_math_grouped_10th_df = student_data_math_df.loc[(student_data_math_df.grade == '10th')].groupby(['school_name'])["math_score"].mean()
student_data_math_grouped_11th_df = student_data_math_df.loc[(student_data_math_df.grade == '11th')].groupby(['school_name'])["math_score"].mean()
student_data_math_grouped_12th_df = student_data_math_df.loc[(student_data_math_df.grade == '12th')].groupby(['school_name'])["math_score"].mean()

student_data_math_grade_df = pd.DataFrame({"9th": student_data_math_grouped_9th_df,
                                             "10th": student_data_math_grouped_10th_df,
                                             "11th": student_data_math_grouped_11th_df,
                                             "12th": student_data_math_grouped_12th_df,
                                            })

format_dict_grades = {
    "9th":"{:,.6f}", 
    "10th":"{:,.6f}", 
    "11th":"{:,.6f}", 
    "12th":"{:,.6f}", 
    }

student_data_math_grade_df = student_data_math_grade_df.reset_index()
student_data_math_grade_df = student_data_math_grade_df.rename(columns = {"school_name": ""})
student_data_math_grade_df = student_data_math_grade_df.style.format(format_dict_grades).hide_index().applymap(bold_cols, subset=pd.IndexSlice[:, ['']])
student_data_math_grade_df


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [92]:
student_data_reading_df = student_data[['school_name','grade','reading_score']]
student_data_reading_grouped_9th_df = student_data_reading_df.loc[(student_data_reading_df.grade == '9th')].groupby(['school_name'])["reading_score"].mean()
student_data_reading_grouped_10th_df = student_data_reading_df.loc[(student_data_reading_df.grade == '10th')].groupby(['school_name'])["reading_score"].mean()
student_data_reading_grouped_11th_df = student_data_reading_df.loc[(student_data_reading_df.grade == '11th')].groupby(['school_name'])["reading_score"].mean()
student_data_reading_grouped_12th_df = student_data_reading_df.loc[(student_data_reading_df.grade == '12th')].groupby(['school_name'])["reading_score"].mean()

student_data_reading_grade_df = pd.DataFrame({"9th": student_data_reading_grouped_9th_df,
                                             "10th": student_data_reading_grouped_10th_df,
                                             "11th": student_data_reading_grouped_11th_df,
                                             "12th": student_data_reading_grouped_12th_df,
                                            })

format_dict_grades = {
    "9th":"{:,.6f}", 
    "10th":"{:,.6f}", 
    "11th":"{:,.6f}", 
    "12th":"{:,.6f}", 
    }

student_data_reading_grade_df = student_data_reading_grade_df.reset_index()
student_data_reading_grade_df = student_data_reading_grade_df.rename(columns = {"school_name": ""})
student_data_reading_grade_df = student_data_reading_grade_df.style.format(format_dict_grades).hide_index().applymap(bold_cols, subset=pd.IndexSlice[:, ['']])
student_data_reading_grade_df


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

Use bins and calculate average maths and reading scores for each bin 

In [71]:
bins = [0,584, 629, 644, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
scores_by_spending_df = school_data_complete[["math_score","reading_score"]].copy()
scores_by_spending_df["per_student_spend"] = school_data_complete["budget"]/school_data_complete["size"]
scores_by_spending_df["spending_ranges"] = pd.cut(scores_by_spending_df["per_student_spend"], bins, labels=group_names, include_lowest=True)
scores_by_spending_grouped_df = scores_by_spending_df.groupby(["spending_ranges"])

scores_by_spending_ranged_df = scores_by_spending_grouped_df.mean()


Calculate percentage pass rate for Maths

In [72]:
total_students_maths_series = scores_by_spending_grouped_df["math_score"].count()
student_maths_passed_df = scores_by_spending_df.loc[scores_by_spending_df.math_score >= 70].groupby(["spending_ranges"]).count()
math_passed_series = student_maths_passed_df["math_score"]
math_percentage_ranged_df = pd.DataFrame((math_passed_series/total_students_maths_series)*100)
math_percentage_ranged_df = math_percentage_ranged_df.rename(columns={"math_score":"% Passing Math"})

Calculate percentage pass rate for reading

In [73]:
total_students_reading_series = scores_by_spending_grouped_df["reading_score"].count()
student_readings_passed_df = scores_by_spending_df.loc[scores_by_spending_df.reading_score >= 70].groupby(["spending_ranges"]).count()
reading_passed_series = student_readings_passed_df["reading_score"]
reading_percentage_ranged_df = pd.DataFrame((reading_passed_series/total_students_reading_series)*100)
reading_percentage_ranged_df = reading_percentage_ranged_df.rename(columns={"reading_score":"% Passing reading"})

Overall Overall Passing  is measured as percentage of students passed both maths and reading having marks >=70 marks

In [74]:
total_students_overall_series = scores_by_spending_grouped_df["math_score"].count()
overall_pass_df = scores_by_spending_df.loc[(scores_by_spending_df.math_score >= 70) & (scores_by_spending_df.reading_score >= 70)].groupby(["spending_ranges"]).count()
overall_pass_series = overall_pass_df["math_score"]

overall_percentage_df = pd.DataFrame((overall_pass_series/total_students_overall_series)*100)
overall_percentage_df = overall_percentage_df.rename(columns = {
    "math_score":"% Overall Passing"
    })

percentage_df=pd.merge(math_percentage_ranged_df,reading_percentage_ranged_df,on="spending_ranges")
percentage_df=pd.merge(percentage_df,overall_percentage_df,on="spending_ranges")
percentage_df=pd.merge(scores_by_spending_ranged_df,percentage_df,on="spending_ranges") 
percentage_df = percentage_df.rename(columns = {"math_score":"Average Math Score",
                      "reading_score" : "Average Reading Score"})
del percentage_df["per_student_spend"]

format_dict_scores = {
    "Average Math Score":"{:,.2f}", 
    "Average Reading Score":"{:,.2f}", 
    "% Passing Math":"{:,.2f}", 
    "% Passing reading":"{:,.2f}", 
    "% Overall Passing":"{:,.2f}"
    }

percentage_df = percentage_df.style.format(format_dict_scores)
percentage_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
spending_ranges,,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,79.98,82.31,79.11,88.51,70.94
$630-644,77.82,81.30,70.62,82.60,58.84
$645-675,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In defining schools as Large, Medium and Small, used the Q1 and Q3 figures of the student count distribution of schools as thresholds in defining them. 

df_dist = school_data[["school_name","size"]]
df_dist.describe()
Q1 = 1698
Q3 = 3474
max = 4976


In [75]:

bins = [0,1698, 3474, 4977]
group_names = ["Small (<1698)", "Medium (1699-3474)", "Large (3475>)"]

scores_by_size_df = school_data_complete[["math_score","reading_score","size"]].copy()
scores_by_size_df["School Size"] = pd.cut(scores_by_size_df["size"], bins, labels=group_names, include_lowest=True)
del scores_by_size_df["size"]
scores_by_size_grouped_df = scores_by_size_df.groupby(["School Size"])
scores_by_size_ranged_df = scores_by_size_grouped_df.mean()


Calculate percentage passed on reading

In [76]:
total_students_maths_series = scores_by_size_grouped_df["math_score"].count()
student_maths_passed_df = scores_by_size_df.loc[scores_by_size_df.math_score >= 70].groupby(["School Size"]).count()
math_passed_series = student_maths_passed_df["math_score"]
math_percentage_ranged_df = pd.DataFrame((math_passed_series/total_students_maths_series)*100)
math_percentage_ranged_df = math_percentage_ranged_df.rename(columns={"math_score":"% Passing Math"})


Calculate percentage passed on reading

In [77]:
total_students_reading_series = scores_by_size_grouped_df["reading_score"].count()
student_readings_passed_df = scores_by_size_df.loc[scores_by_size_df.reading_score >= 70].groupby(["School Size"]).count()
reading_passed_series = student_readings_passed_df["reading_score"]
reading_percentage_ranged_df = pd.DataFrame((reading_passed_series/total_students_reading_series)*100)
reading_percentage_ranged_df = reading_percentage_ranged_df.rename(columns={"reading_score":"% Passing reading"})

Calculate % passed both maths and reading based on school size

In [79]:
total_students_overall_series = scores_by_size_grouped_df["math_score"].count()
overall_pass_df = scores_by_size_df.loc[(scores_by_size_df.math_score >= 70) & (scores_by_size_df.reading_score >= 70)].groupby(["School Size"]).count()
overall_pass_series = overall_pass_df["math_score"]

overall_percentage_df = pd.DataFrame((overall_pass_series/total_students_overall_series)*100)
overall_percentage_df = overall_percentage_df.rename(columns = {
    "math_score":"% Overall Passing"
    })

percentage_df=pd.merge(math_percentage_ranged_df,reading_percentage_ranged_df,on="School Size")
percentage_df=pd.merge(percentage_df,overall_percentage_df,on="School Size")
percentage_df=pd.merge(scores_by_size_ranged_df,percentage_df,on="School Size") 
percentage_df = percentage_df.rename(columns = {"math_score":"Average Math Score",
                      "reading_score" : "Average Reading Score"})

format_dict_scores = {
    "Average Math Score":"{:,.2f}", 
    "Average Reading Score":"{:,.2f}", 
    "% Passing Math":"{:,.2f}", 
    "% Passing reading":"{:,.2f}", 
    "% Overall Passing":"{:,.2f}"
    }

percentage_df = percentage_df.style.format(format_dict_scores)
percentage_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
School Size,,,,,
Small (<1698),83.52,83.88,93.52,96.86,90.58
Medium (1699-3474),79.89,82.40,79.46,88.05,71.08
Large (3475>),77.07,80.93,66.47,81.11,53.72


## Scores by School Type

* Perform the same operations as above, based on school type

Calculate maths and reading average scores based on School Type

In [80]:
scores_by_type_df = school_data_complete[["math_score","reading_score","type"]].copy()
scores_by_type_df = scores_by_type_df.rename(columns = {"type":"School Type"})
scores_by_type_grouped_df = scores_by_type_df.groupby(["School Type"])
scores_by_type_ranged_df = scores_by_type_grouped_df.mean()

Calculate % passing rate of maths

In [81]:
total_students_maths_series = scores_by_type_grouped_df["math_score"].count()
student_maths_passed_df = scores_by_type_df.loc[scores_by_type_df.math_score >= 70].groupby(["School Type"]).count()
math_passed_series = student_maths_passed_df["math_score"]
math_percentage_ranged_df = pd.DataFrame((math_passed_series/total_students_maths_series)*100)
math_percentage_ranged_df = math_percentage_ranged_df.rename(columns={"math_score":"% Passing Math"})


Calculate % passing rate of reading

In [85]:
total_students_reading_series = scores_by_type_grouped_df["reading_score"].count()
student_readings_passed_df = scores_by_type_df.loc[scores_by_type_df.reading_score >= 70].groupby(["School Type"]).count()
reading_passed_series = student_readings_passed_df["reading_score"]
reading_percentage_ranged_df = pd.DataFrame((reading_passed_series/total_students_reading_series)*100)
reading_percentage_ranged_df = reading_percentage_ranged_df.rename(columns={"reading_score":"% Passing reading"})

Calculate % passing of both maths and reading 

In [84]:
total_students_overall_series = scores_by_type_grouped_df["math_score"].count()
overall_pass_df = scores_by_type_df.loc[(scores_by_type_df.math_score >= 70) & (scores_by_type_df.reading_score >= 70)].groupby(["School Type"]).count()
overall_pass_series = overall_pass_df["math_score"]

overall_percentage_df = pd.DataFrame((overall_pass_series/total_students_overall_series)*100)
overall_percentage_df = overall_percentage_df.rename(columns = {
    "math_score":"% Overall Passing"
    })


percentage_df=pd.merge(math_percentage_ranged_df,reading_percentage_ranged_df,on="School Type")
percentage_df=pd.merge(percentage_df,overall_percentage_df,on="School Type")
percentage_df=pd.merge(scores_by_type_ranged_df,percentage_df,on="School Type") 
percentage_df = percentage_df.rename(columns = {"math_score":"Average Math Score",
                      "reading_score" : "Average Reading Score"})

format_dict_scores = {
    "Average Math Score":"{:,.2f}", 
    "Average Reading Score":"{:,.2f}", 
    "% Passing Math":"{:,.2f}", 
    "% Passing reading":"{:,.2f}", 
    "% Overall Passing":"{:,.2f}"
    }

percentage_df = percentage_df.style.format(format_dict_scores)
percentage_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing
School Type,,,,,
Charter,83.41,83.90,93.70,96.65,90.56
District,76.99,80.96,66.52,80.91,53.70
